In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：用于在线预测的自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_online.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

概况

本教程演示如何使用Vertex SDK来训练和部署用于在线预测的自定义图像分类模型。

数据集

本教程使用的数据集是来自TensorFlow数据集的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已内置于TensorFlow中。训练模型预测图像属于十种类别中的哪一类：飞机，汽车，鸟，猫，鹿，狗，青蛙，马，船，卡车。

### 目标

在本教程中，您将使用Vertex SDK在Google预构建的Docker容器中的Python脚本中创建一个自定义模型，然后通过发送数据对部署的模型进行预测。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建自定义模型。

执行的步骤包括：

- 为训练模型创建一个Vertex自定义作业。
- 训练一个TensorFlow模型。
- 检索和加载模型文件。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成成本估算。

### 设置您的本地开发环境

如果您正在使用Colab或Google Cloud Notebooks，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简洁的一组说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端中的命令行上运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端中的命令行上运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开这个笔记本。

安装

安装最新版本的Python的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新版的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

重新启动内核

安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

这个教程不需要 GPU 运行时。

### 设置你的 Google 云项目

**无论你是在哪个笔记本环境中，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当你首次创建一个帐户时，你会获得 $300 的免费信用，用于支付计算/存储费用。

2. [确保你的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API： Vertex AI APIs, Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果你是在本地运行这个笔记本，你需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入你的项目 ID。然后运行这个单元格，以确保 Cloud SDK 在这个笔记本中的所有命令中都使用正确的项目。

**注意：** Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并且以 `$` 为前缀的 Python 变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

您还可以更改`REGION`变量，该变量用于在本笔记本的其余部分进行操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有地区都为所有Vertex AI服务提供支持。

了解更多有关[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，可能会使用共享测试账号或项目。为了避免在创建资源时发生名称冲突，您可以为每个实例会话创建一个时间戳，并将该时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经通过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth认证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在 **授予此服务帐号对项目的访问权限** 部分，点击角色下拉列表。在过滤框中输入“Vertex”，选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您在哪个笔记本环境中，都需要遵循以下步骤。**

当您初始化Python的Vertex SDK时，需要指定一个云存储的暂存桶。这个暂存桶是您的数据集和模型资源在不同会话之间保留的地方。

在下面设置您的云存储桶的名称。桶名必须在所有谷歌云项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python版本的Vertex SDK

为您的项目和相应的存储桶初始化Python版本的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持 GPU 的容器映像和分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 分配给每个 VM 的 GPU 容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，指定`(None，None)`以使用一个在 CPU 上运行的容器映像。

在您的地区学习更多[关于](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)硬件加速器支持。

*注意*：早于 2.3 版本的 TF 对 GPU 支持将在此教程中加载自定义模型时失败。这是一个已知问题，并在 TF 2.3 中得到修复——此问题是由提供功能中生成的静态图操作所致。如果在您自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器映像。

有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 个

*注意：以下机器类型不支持用于训练：*

 - `standard`: 2 个 vCPUs
 - `highcpu`: 2、4 和 8 个 vCPUs

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好了，可以开始创建你自己的定制模型并为CIFAR10进行训练。

### 检查培训套件

#### 套件布局

在开始培训之前，您将查看如何为自定义培训工作装配 Python 套件。当解压缩时，套件包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是安装套件到 Docker 镜像操作环境的指令。

文件 `trainer/task.py` 是执行自定义培训工作的 Python 脚本。*注意*，在工作池规范中我们提到时，我们将目录斜杠替换为点号 (`trainer.task`) 并删除文件后缀 (`.py`)。

#### 套件装配

在下面的单元格中，您将装配培训套件。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py的内容

在下一个单元格中，您要编写训练脚本task.py的内容。我们不会详细讨论，只是让您浏览一下。总结：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果没有指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TF数据集（tfds）中加载CIFAR10数据集。
- 使用TF.Keras模型API构建模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分发策略。
- 根据参数`args.epochs`和`args.steps`训练模型（`fit()`）。
- 将经过训练的模型保存（`save(args.model_dir)`）到指定的模型目录。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶上

接下来，您将培训文件夹打包成一个压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，和2）运行该任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，具有以下参数：

- `display_name`：自定义训练任务的易读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="cifar10_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`: 传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：在我们的演示中，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接方式：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量 `DIRECT = True`），或
      - 间接方式：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的周期数。
  - `"--steps=" + STEPS`：每个周期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

执行自定义训练作业

接下来，通过调用方法`run`运行自定义作业来启动训练作业，具有以下参数：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作人员复制品的加速器数量。
- `base_output_dir`：要将模型工件写入的Cloud Storage位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶中加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的云存储路径传递给它，该路径由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在来评估模型的表现。

### 加载评估数据

您将从`tf.keras.datasets`加载CIFAR10测试（留存）数据，使用`load_data()`方法。这会将数据集作为两个元素的元组返回。第一个元素是训练数据，第二个是测试数据。每个元素也是两个元素的元组：图像数据和对应的标签。

您不需要训练数据，所以我们以`(_, _)`的方式加载它。

在运行数据进行评估之前，您需要对其进行预处理：

`x_test`：
1. 通过将每个像素除以255来对像素数据进行规范化（重新缩放）。这将每个单字节整数像素替换为介于0和1之间的32位浮点数。

`y_test`：<br/>
2. 标签当前是标量（稀疏的）。如果您回顾`trainer/task.py`脚本中的`compile()`步骤，您会发现它是为稀疏标签而编译的。因此我们不需要做更多事情。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估定制工作中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

###图像数据的serving功能

要将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为base 64 -- 这样在通过网络传输二进制数据时，使内容免受修改。由于此部署模型期望将输入数据作为原始（未压缩）字节传递，因此您需要确保base 64编码的数据在传递给部署模型的输入之前被转换回原始字节。

为此，请定义一个serving功能（`serving_fn`），并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便serving函数与底层模型融合（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求的内容会被base 64解码为Tensorflow字符串（`tf.string`），然后传递给serving函数（`serving_fn`）。服务函数将“ tf.string”预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以满足模型的输入要求：
- `io.decode_jpeg` - 解压缩JPG图像，返回三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float 32。
- `image.resize` - 调整图像尺寸以匹配模型的输入形状。
- `resized / 255.0` - 重新缩放（标准化）像素数据在0和1之间。

此时，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

获取服务功能的签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务功能的签名。为什么呢？当我们将数据作为HTTP请求包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的服务功能将从base64转换为numpy数组。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 - 这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到`Model`资源，参数如下：

- `display_name`: `Model`资源的可读名称。
- `artifact`: 训练模型文件的云存储位置。
- `serving_container_image_uri`: 用于提供服务的容器映像。
- `sync`: 是否异步执行上传。

如果`upload()`方法是异步运行的，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aip.Model.upload(
    display_name="cifar10_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

## 部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法，并提供以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：在端点上流量的百分比分配给这个模型，以字典形式指定一个或多个键/值对。
如果只有一个模型，则指定为{ "0"：100 }，其中"0"指的是上传的这个模型，100表示100%的流量。
如果在端点上存在其他模型，需要分配流量，则使用model_id指定为{ "0"：percent，model_id：percent，... }，其中model_id是已部署到端点的现有模型的模型ID。各百分比必须累加为100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作实例的加速器的数量。
- `starting_replica_count`：初始预留的计算实例数量。
- `max_replica_count`：可扩展到的最大计算实例数量。在本教程中，仅预留了一个实例。

In [ ]:
DEPLOYED_NAME = "cifar10-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

获取测试项

您将使用数据集中测试（留存）部分的一个示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

### 准备请求内容
您将发送 CIFAR10 图像作为压缩的 JPG 图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用 OpenCV 将未压缩的图像写入磁盘作为压缩的 JPEG 图像。
 - 将图像数据从 \[0,1) 范围反归一化为 [0,255)。
 - 将 32 位浮点值转换为 8 位无符号整数。
- `tf.io.read_file`：将压缩的 JPG 图像作为原始字节读取到内存中。
- `base64.b64encode`：将原始字节编码为一个 base 64 编码的字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

在这个示例中，您的测试项目位于一个 Cloud Storage 存储桶中，您可以使用 `tf.io.gfile.Gfile()` 打开并读取图像的内容。为了将测试数据传递给预测服务，您将字节编码为 base64 格式，这样可以使内容在网络传输时安全免受修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），因此请将您的单个测试项目作为一个项目列表发送。

#### 响应

从 `predict()` 调用返回的是一个 Python 字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的部署的 `Model` 资源的 Vertex AI 标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

##  卸载模型

当您完成预测之后，您可以从“终端点”资源中卸载模型。这将取消所有计算资源，并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud
项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage 存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME